In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:
# lieux_all = pd.read_csv("lieux.csv")
# vehicules_all = pd.read_csv("vehi.csv")
caracteristiques_all = pd.read_csv("../raw_data/carac.csv")

/var/folders/jw/qw353j7x7h970k1t74h_j5nc0000gn/T/ipykernel_13968/149342845.py:3: DtypeWarning: Columns (4,10,12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  caracteristiques_all = pd.read_csv("../raw_data/carac.csv")


In [3]:
caracteristiques_all.isna().sum()

Num_Acc          55302
an                   0
mois                 0
jour                 0
hrmn                 0
lum                  0
agg                  0
int                  0
atm                 73
col                 19
com                  2
adr             144509
gps             696821
lat             487068
long            487072
dep                  0
Accident_Id    1121571
dtype: int64

# Chargement des données CSV

In [4]:
lieux_df = pd.read_csv('../clean_data/lieux_clean.csv')
usagers_df = pd.read_csv('../clean_data/usagers_clean.csv')
vehicules_df = pd.read_csv('../clean_data/vehicules_clean.csv')
caracteristiques_df = pd.read_csv('../clean_data/caracteristiques_clean.csv')

/var/folders/jw/qw353j7x7h970k1t74h_j5nc0000gn/T/ipykernel_13968/4187407229.py:4: DtypeWarning: Columns (6,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  caracteristiques_df = pd.read_csv('../clean_data/caracteristiques_clean.csv')


In [5]:
caracteristiques_df.isna().sum()

caracteristiques_df[caracteristiques_df['Num_Acc'].isna()]

,Num_Acc,lum,agg,int,atm,col,com,adr,lat,long,dep,date
116623,NaN,1,2,3,1.0,3.0,26198,TEIL(vieille route du),"44,5594200000","4,7257200000",26,NaN
116624,NaN,1,2,3,1.0,3.0,25204,Miranda,"46,9258100000","6,3462000000",25,NaN
116625,NaN,1,2,6,1.0,2.0,22360,ROND POINT DE BREZILLET,"48,4931620000","-2,7604390000",22,NaN
116626,NaN,1,2,3,8.0,6.0,16102,LOHMEYER (RUE),"45,6926520000","-0,3262900000",16,NaN
116627,NaN,1,1,1,1.0,2.0,13103,ROUTE DE JEAN MOULIN-RN 538,"43,6755790366","5,0927031775",13,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
171920,NaN,3,1,1,1.0,6.0,2B293,D71,"42,3101650000","9,4785830000",2B,NaN
171921,NaN,3,1,1,1.0,6.0,84074,D973,"43,7531640000","5,2244760000",84,NaN
171922,NaN,3,1,1,9.0,6.0,74001,D22,"46,2825320000","6,7328060000",74,NaN
171923,NaN,1,1,3,1.0,3.0,81099,Chemin Toulze,"43,9272650000","1,9156370000",81,NaN


In [6]:
# fonction rolliwn window ? applique une fonction sur une partie précise


# Merge de la données


In [7]:
# Création d'une colonne uniques pour merger les df usagers et véhicules

usagers_df['aug'] = usagers_df['Num_Acc'].astype(str) + usagers_df['num_veh'].astype(str)
vehicules_df['aug'] = vehicules_df['Num_Acc'].astype(str) + vehicules_df['num_veh'].astype(str)


In [8]:
vehicules_df.shape
vehicules_df = vehicules_df[vehicules_df['catv']==1]

In [9]:
# On drop les duplicates lorsqu'il y a une plusieurs usagers dans un même véhicules et pour le même accident
# On crée une colonne pour y ajouter le nombre d'usagers par véhicules


usagers_df['dup_count'] = usagers_df.groupby('aug')['aug'].transform('count')
usagers_df =usagers_df.drop_duplicates(subset=['aug'], keep='first')

In [10]:
print(type(vehicules_df))
print(type(usagers_df))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [11]:
# On merge véhicules et usagers puis on ne garde que les accidents impliquant un vélo
# On clean et on drop les doublons

vehi_usa = pd.merge(vehicules_df, usagers_df, on='aug', how='inner')
vehi_usa = vehi_usa.drop(columns=['num_veh_x', 'num_veh_y', 'Num_Acc_y'])
vehi_usa = vehi_usa.rename(columns={'Num_Acc_x': 'Num_Acc'})


In [12]:
vehi_usa['dup_count'].value_counts()

1    83223
2     5132
3      156
4        5
6        3
5        2
Name: dup_count, dtype: int64

In [13]:
print(vehi_usa['Num_Acc'].duplicated().any())
print(caracteristiques_df['Num_Acc'].duplicated().any())


True
True


In [14]:
duplicates_velo = vehi_usa[vehi_usa['Num_Acc'].duplicated(keep=False)]
duplicates_velo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5148 entries, 12 to 88515
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Num_Acc    5148 non-null   int64  
 1   catv       5148 non-null   int64  
 2   obs        5148 non-null   float64
 3   obsm       5148 non-null   float64
 4   choc       5148 non-null   float64
 5   manv       5148 non-null   float64
 6   aug        5148 non-null   object 
 7   grav       5148 non-null   int64  
 8   sexe       5148 non-null   int64  
 9   trajet     5148 non-null   int64  
 10  secu       5148 non-null   float64
 11  locp       5148 non-null   float64
 12  actp       5148 non-null   int64  
 13  etatp      5148 non-null   float64
 14  an_nais    5148 non-null   float64
 15  dup_count  5148 non-null   int64  
dtypes: float64(8), int64(7), object(1)
memory usage: 683.7+ KB


In [15]:
# On merge les df caractéristiques et lieux

velo_df = pd.merge(vehi_usa, lieux_df, on='Num_Acc', how='left')
velo_df = pd.merge(velo_df, caracteristiques_df, on='Num_Acc', how='left')


In [16]:
velo_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88521 entries, 0 to 88520
Data columns (total 36 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Num_Acc    88521 non-null  int64  
 1   catv       88521 non-null  int64  
 2   obs        88521 non-null  float64
 3   obsm       88521 non-null  float64
 4   choc       88521 non-null  float64
 5   manv       88521 non-null  float64
 6   aug        88521 non-null  object 
 7   grav       88521 non-null  int64  
 8   sexe       88521 non-null  int64  
 9   trajet     88521 non-null  int64  
 10  secu       88521 non-null  float64
 11  locp       88521 non-null  float64
 12  actp       88521 non-null  int64  
 13  etatp      88521 non-null  float64
 14  an_nais    88521 non-null  float64
 15  dup_count  88521 non-null  int64  
 16  catr       88521 non-null  float64
 17  circ       88521 non-null  float64
 18  nbv        88521 non-null  float64
 19  vosp       88521 non-null  float64
 20  prof  

In [17]:
# On remplace les valeurs manquantes par la valeur la plus fréquente
velo_df['lum'].fillna(velo_df['lum'].mode()[0], inplace=True)
velo_df['agg'].fillna(velo_df['agg'].mode()[0], inplace=True)
velo_df['atm'].fillna(velo_df['atm'].mode()[0], inplace=True)
velo_df['atm'] = velo_df['atm'].replace({ -1: 1, 9: 1})

In [18]:
# Obtenez la distribution des valeurs
distribution = velo_df['int'].value_counts(normalize=True)
# Obtenez les valeurs manquantes
missing = velo_df['int'].isnull()
# Remplissez les valeurs manquantes en fonction de la distribution
velo_df.loc[missing, 'int'] = np.random.choice(distribution.index, size=len(velo_df[missing]), p=distribution.values)

In [19]:
# Obtenez la distribution des valeurs
distribution = velo_df['col'].value_counts(normalize=True)
# Obtenez les valeurs manquantes
missing = velo_df['col'].isnull()
# Remplissez les valeurs manquantes en fonction de la distribution
velo_df.loc[missing, 'col'] = np.random.choice(distribution.index, size=len(velo_df[missing]), p=distribution.values)
# Je remplace le -1 par 1
velo_df['col'] = velo_df['col'].replace({ -1: 3})

In [20]:
velo_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88521 entries, 0 to 88520
Data columns (total 36 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Num_Acc    88521 non-null  int64  
 1   catv       88521 non-null  int64  
 2   obs        88521 non-null  float64
 3   obsm       88521 non-null  float64
 4   choc       88521 non-null  float64
 5   manv       88521 non-null  float64
 6   aug        88521 non-null  object 
 7   grav       88521 non-null  int64  
 8   sexe       88521 non-null  int64  
 9   trajet     88521 non-null  int64  
 10  secu       88521 non-null  float64
 11  locp       88521 non-null  float64
 12  actp       88521 non-null  int64  
 13  etatp      88521 non-null  float64
 14  an_nais    88521 non-null  float64
 15  dup_count  88521 non-null  int64  
 16  catr       88521 non-null  float64
 17  circ       88521 non-null  float64
 18  nbv        88521 non-null  float64
 19  vosp       88521 non-null  float64
 20  prof  

In [21]:
velo_df = velo_df[~(velo_df.adr.isna() & velo_df.lat.isna() & velo_df.long.isna() & velo_df.com.isna() & velo_df.dep.isna())]

In [22]:
velo_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83233 entries, 0 to 88520
Data columns (total 36 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Num_Acc    83233 non-null  int64  
 1   catv       83233 non-null  int64  
 2   obs        83233 non-null  float64
 3   obsm       83233 non-null  float64
 4   choc       83233 non-null  float64
 5   manv       83233 non-null  float64
 6   aug        83233 non-null  object 
 7   grav       83233 non-null  int64  
 8   sexe       83233 non-null  int64  
 9   trajet     83233 non-null  int64  
 10  secu       83233 non-null  float64
 11  locp       83233 non-null  float64
 12  actp       83233 non-null  int64  
 13  etatp      83233 non-null  float64
 14  an_nais    83233 non-null  float64
 15  dup_count  83233 non-null  int64  
 16  catr       83233 non-null  float64
 17  circ       83233 non-null  float64
 18  nbv        83233 non-null  float64
 19  vosp       83233 non-null  float64
 20  prof  

In [23]:
# Réinitialisez l'index de votre DataFrame
velo_df = velo_df.reset_index(drop=True)

# Obtenez la distribution des valeurs
distribution = velo_df['date'].value_counts(normalize=True)

# Obtenez les valeurs manquantes
missing = velo_df['date'].isnull()

# Remplissez les valeurs manquantes en fonction de la distribution
velo_df.loc[missing, 'date'] = np.random.choice(distribution.index, size=len(velo_df[missing]), p=distribution.values)

In [24]:
velo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83233 entries, 0 to 83232
Data columns (total 36 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Num_Acc    83233 non-null  int64  
 1   catv       83233 non-null  int64  
 2   obs        83233 non-null  float64
 3   obsm       83233 non-null  float64
 4   choc       83233 non-null  float64
 5   manv       83233 non-null  float64
 6   aug        83233 non-null  object 
 7   grav       83233 non-null  int64  
 8   sexe       83233 non-null  int64  
 9   trajet     83233 non-null  int64  
 10  secu       83233 non-null  float64
 11  locp       83233 non-null  float64
 12  actp       83233 non-null  int64  
 13  etatp      83233 non-null  float64
 14  an_nais    83233 non-null  float64
 15  dup_count  83233 non-null  int64  
 16  catr       83233 non-null  float64
 17  circ       83233 non-null  float64
 18  nbv        83233 non-null  float64
 19  vosp       83233 non-null  float64
 20  prof  

In [25]:
velo_df.to_csv('../clean_data/velo_df.csv', index=False)